In [1]:
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
intents = json.loads(open("intents.json").read())

In [4]:
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))
model = load_model("chatbot_model.h5")

In [5]:
def cleanup(sentence):
  sentenceWords = nltk.word_tokenize(sentence)
  sentenceWords = [lemmatizer.lemmatize(word.lower()) for word in sentenceWords]
  return sentenceWords

In [6]:
def bagOfWords(sentence):
  sentenceWords = cleanup(sentence)
  bag = [0] * len(words)
  for w in sentenceWords:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1

  return np.array(bag)

In [7]:
def predict(sentence):
  bow = bagOfWords(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i , r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key = lambda x : x[1], reverse = True)
  returnList = []
  for r in results:
    returnList.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return returnList   

In [8]:
def getResponse(intentsList, intentsjson):
  tag = intentsList[0]['intent']
  listOfIntents = intentsjson['intents']
  for i in listOfIntents:
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result 

In [11]:
print("Bot is running")

while True:
  message = input("")
  if message == 'thanks':
    break

  ints = predict(message)
  res = getResponse(ints, intents)
  print(res)

Bot is running
greetings
Hey!I am Alan, your virtual assistant
how are you
Hey!I am Alan, your virtual assistant
thanks
